In [12]:
import json
with open('./credentials.json', 'r') as creds:
  credentials = json.load(creds)

In [13]:
SAP_HANA_CLOUD_ADDRESS  = credentials["SAP_HANA_CLOUD"]["HANA_ADDRESS"]
SAP_HANA_CLOUD_PORT     = credentials["SAP_HANA_CLOUD"]["HANA_PORT"]
SAP_HANA_CLOUD_USER     = credentials["SAP_HANA_CLOUD"]["HANA_USER"]
SAP_HANA_CLOUD_PASSWORD = credentials["SAP_HANA_CLOUD"]["HANA_PASSWORD"]

In [14]:
import os
os.environ["AICORE_CLIENT_ID"]      = credentials["SAP_AI_CORE"]["AICORE_CLIENT_ID"]
os.environ["AICORE_CLIENT_SECRET"]  = credentials["SAP_AI_CORE"]["AICORE_CLIENT_SECRET"]
os.environ["AICORE_AUTH_URL"]       = credentials["SAP_AI_CORE"]["AICORE_AUTH_URL"]
os.environ["AICORE_RESOURCE_GROUP"] = credentials["SAP_AI_CORE"]["AICORE_RESOURCE_GROUP"]
os.environ["AICORE_BASE_URL"]       = credentials["SAP_AI_CORE"]["AICORE_BASE_URL"]   

In [15]:
import hana_ml.dataframe as dataframe
conn = dataframe.ConnectionContext(
                                   address  = SAP_HANA_CLOUD_ADDRESS,
                                   port     = SAP_HANA_CLOUD_PORT,
                                   user     = SAP_HANA_CLOUD_USER,
                                   password = SAP_HANA_CLOUD_PASSWORD, 
                                  )
conn.connection.isconnected()

True

In [16]:
df_remote_toprocess = conn.sql('''SELECT "AID", "QID", "QUESTION" FROM FAQ_QUESTIONS WHERE QUESTION_VECTOR IS NULL ORDER BY "AID", "QID" ''')


In [17]:
df_remote_toprocess.head(5).collect()


,AID,QID,QUESTION


In [18]:
df_remote_toprocess.count()


0

In [19]:
from gen_ai_hub.proxy.langchain.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(proxy_model_name='text-embedding-ada-002')

In [20]:
dbapi_cursor = conn.connection.cursor()
rowids_toprocess = df_remote_toprocess.select("AID", "QID", "QUESTION").collect()
for index, row_toprocess in rowids_toprocess.iterrows(): 
    my_embedding = embedding.embed_documents(row_toprocess['QUESTION']) 
    my_embedding_str = str(my_embedding[0])
    my_aid = row_toprocess['AID']
    my_qid = row_toprocess['QID']
    print(str(my_aid) + '-' + str(my_qid) + ': ' + str(my_embedding_str[:100]))
    dbapi_cursor.execute(f"""UPDATE "FAQ_QUESTIONS" SET "QUESTION_VECTOR" = TO_REAL_VECTOR('{my_embedding_str}') 
                             WHERE "AID" = {my_aid} AND "QID" = {my_qid};""")  

In [21]:
df_remote = conn.table('FAQ_QUESTIONS').sort(['AID', 'QID'])
df_remote.head(5).collect()

,AID,QID,QUESTION,QUESTION_VECTOR
0,1000,1,When was SAP founded?,"[0, 6, 0, 0, 181, 99, 141, 60, 145, 136, 143, ..."
1,1001,1,"What does the acronym ""SAP"" stand for?","[0, 6, 0, 0, 181, 99, 141, 60, 145, 136, 143, ..."
2,1002,1,What is SAP’s vision and mission?,"[0, 6, 0, 0, 181, 99, 141, 60, 145, 136, 143, ..."
3,1003,1,What is the business outlook for the current f...,"[0, 6, 0, 0, 181, 99, 141, 60, 145, 136, 143, ..."
4,1004,1,Are you planning to grow organically or throug...,"[0, 6, 0, 0, 44, 114, 124, 60, 54, 164, 97, 18..."
